In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [2]:
df = pd.read_csv(r"C:\Users\maill\fusion3_clean.csv", low_memory=False)
df.drop(['Unnamed: 0', 'num_acc'],axis=1, inplace=True)
df = df.dropna()

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

In [4]:
# Séparation de la variable cible y avant l'encodage
y = df['grav']

X = df.drop('grav', axis=1)


In [5]:
# Création de l'ensemble d'entrainement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
X_train.columns

Index(['place', 'catu', 'sexe', 'trajet', 'locp', 'actp', 'etatp', 'secuUn',
       'tranches_ages', 'catr', 'circ', 'nbv', 'vosp', 'prof', 'plan', 'surf',
       'infra', 'situ', 'obs', 'obsm', 'choc', 'manv', 'catv_Label', 'lum',
       'agg', 'int', 'atm', 'col', 'com', 'dep', 'jour_de_la_semaine',
       'heure'],
      dtype='object')

In [7]:
for col in X_train:
    # Calculer la fréquence de chaque catégorie
    frequency_encoding = X_train[col].value_counts(normalize=True)

    # Remplacer chaque catégorie par sa fréquence
    X_train[col] = X_train[col].map(frequency_encoding)
    
for col in X_test:
    # Calculer la fréquence de chaque catégorie
    frequency_encoding = X_test[col].value_counts(normalize=True)

    # Remplacer chaque catégorie par sa fréquence
    X_test[col] = X_test[col].map(frequency_encoding)

In [8]:
from sklearn.utils.class_weight import compute_class_weight

# Étape 1 : Calculer les poids des classes
classes = ['Blessé léger', 'Indemne', 'Blessé hospitalisé', 'Tué']
class_weights = compute_class_weight('balanced', classes=classes, y=y_train)

# Mapping des noms de classe aux poids
class_weight_dict = {class_label: weight for class_label, weight in zip(classes, class_weights)}

# Étape 2 : Créer un tableau de poids d'échantillons
sample_weights = np.array([class_weight_dict[label] for label in y_train])




In [9]:
# Entrainement du modèle
from time import time
from sklearn.ensemble import GradientBoostingClassifier

t0 = time()

clf = GradientBoostingClassifier(max_depth=25, n_iter_no_change=10, validation_fraction=0.1, tol=0.01)
clf.fit(X_train, y_train, sample_weight=sample_weights)

t1 = time() - t0
print("Réalisé en {} secondes".format(round(t1,3)))

Réalisé en 51933.872 secondes


In [10]:
# Evaluation du modèle
y_pred = clf.predict(X_test)

print(pd.crosstab(y_test, y_pred, rownames=['Classe réelle'], colnames=['Classe prédite']))
print(classification_report(y_test, y_pred))


Classe prédite      Blessé hospitalisé  Blessé léger  Indemne   Tué
Classe réelle                                                      
Blessé hospitalisé               46072         28785    12651  6136
Blessé léger                     31652         93353    37306  2731
Indemne                          13499         27617   144456  1903
Tué                               6212          2243     1206  2526
                    precision    recall  f1-score   support

Blessé hospitalisé       0.47      0.49      0.48     93644
      Blessé léger       0.61      0.57      0.59    165042
           Indemne       0.74      0.77      0.75    187475
               Tué       0.19      0.21      0.20     12187

          accuracy                           0.62    458348
         macro avg       0.50      0.51      0.51    458348
      weighted avg       0.62      0.62      0.62    458348



In [11]:
f1_micro = f1_score(y_test, y_pred, average='micro')

print(f'F1-Score micro: {f1_micro}')

F1-Score micro: 0.6248680042238648
